In [1]:
from konlpy.tag import Okt
import pandas as pd
from collections import Counter

In [2]:
okt = Okt()

In [3]:
# 디카시와, 기존 데이터셋에서 key_word를 추출합니다.
dica_poem = pd.read_csv('/opt/ml/data/dica_poem.csv')
base_poem = pd.read_csv('/opt/ml/data/poem_75percent_add_emotion.csv')
base_poem2 = pd.read_csv('/opt/ml/data/sinchoon+nonpub+400poems+emotion.csv')

In [4]:
len(base_poem)

15884

In [5]:
base_poem = pd.concat([base_poem,base_poem2]).reset_index(drop=True)

In [7]:
len(base_poem['text'])

17033

In [9]:
base_poem = base_poem.drop_duplicates(['text']).reset_index(drop=True)

In [10]:
len(base_poem)

16953

In [3]:
# 키워드를 뽑는 함수
def get_top3_noun(text):
    noun = okt.nouns(text)
    count = Counter(noun)
    noun_list = count.most_common(3)
    rtn_noun = list(map(lambda x: x[0],noun_list))
    return ', '.join(rtn_noun)


In [4]:
text = '마침내 양은 사라졌다한 의식을 잃고서 나는 은주발에 담긴 눈*이 되고 싶었다 갈피가 다 바랜 경전 속에 없던 신이밑창 닳아 낮아진 가죽신 아래에서 흘렀다'

In [5]:
k = get_top3_noun(text)
print(k)

신, 마침내, 양은


In [12]:
# 키워드 추출
dica_keyword = dica_poem['poem'].apply(get_top3_noun)
base_keyword = base_poem['text'].apply(get_top3_noun)

In [13]:
# 생성된 키워드들을 저장
base_poem['key_word'] = base_keyword
dica_poem['key_word'] = dica_keyword

In [14]:
# 디카시의 시 데이터는 poem 컬럼에
# 기존의 시 데이터는 text에 저장되어 있기 때문에
# 새로 데이터 프레임을 만들어 저장합니다.
bp = pd.DataFrame(data={
    'text' : base_poem['text'],
    'key_word': base_poem['key_word']
    })

dp = pd.DataFrame(data={
    'text' : dica_poem['poem'],
    'key_word': dica_poem['key_word']
    })

In [15]:
# 생성된 데이터 프레임을 저장합니다.
add_poem_data = pd.concat([bp,dp]).reset_index(drop=True)
add_poem_data.to_csv('final_poem_data_with_keyword.csv',index=False)